In [1]:
import MDAnalysis as mda
from MDAnalysis.analysis import rms
import numpy as np

# The crystal structure of SARS-COV-2 Spike Protein and its mutant N501Y

We load the two pdb files the **7NEH** for the ordiginal structure and the **7NEG** for the N501Y mutant receptor structure.

In [2]:
u_original = mda.Universe('../data/7neh.pdb')
u_mutant = mda.Universe('../data/7neg.pdb')

We hold the atoms of the chain E (molecule id:3), which represents the **receptor binding domain of SARS-2 COV-2 Spike Glycoprotein** for both structures.

In [3]:
E_original = u_original.select_atoms('segid E and resid 332:527')

print('There are ',E_original.n_atoms, ' atoms, and ',
      E_original.n_residues,' residues in the chain E (Spike Glycoprotein) of the "7NEH" strucrure.')

There are  1551  atoms, and  196  residues in the chain E (Spike Glycoprotein) of the "7NEH" strucrure.


The original Spike Glycoprotein (Chain E), of the '7NEH' pdb structure, has **1551 atoms** and by selcting the `n_residues` command  or only the $C_{\alpha}$ atoms we confirm that it has **195 residues**. From the inspection of the pdb file we know that there are 9 residues that are missing from the chain E. More specifically, there are 8 residues missing form the beginning, with residue numbers 324-331 and 1 residue missing fro the ending of the chain with residue number 528.

In [4]:
E_mutant = u_mutant.select_atoms('segid E and resid 334:516')

print('There are ', E_mutant.n_atoms, ' atoms, and ',
      E_mutant.n_residues,' residues in the chain E (Spike Glycoprotein) of the "7NEG" strucrure.')

There are  1474  atoms, and  183  residues in the chain E (Spike Glycoprotein) of the "7NEG" strucrure.


On the other hand the mutant Spike Glycoprotein (Chain E), of the '7NEG' pdb structure, has less atoms ,**1474 atoms** than the original, and also fewer residues, **183 residues**. This difference in the number of residues is expected since we know that there are 27 missing residues in the chain E of the mutant structure. More specifically, there are 15 missing residues from the beginning (residue numbers: 319-334) and 12 missing residues form the ending of the chain (residue numbers: 517-528).

In order to caclulate the RMSD of the two structures, we have to select their common atoms. First, we select the common residues that are those with residue numbers: 334-516:

In [5]:
original = u_original.select_atoms('segid E and resid 334:516')
mutant = u_mutant.select_atoms('segid E and resid 334:516')

print(f'There are {original.n_atoms} atoms in the orginal structure, and {mutant.n_atoms} atoms in the mutant structure.' )

There are 1462 atoms in the orginal structure, and 1474 atoms in the mutant structure.


We observe that the structures still have different number of atoms, which is expected, since some atoms might not have been detected in the two methods of extraction. To that end, we have to find their common atoms. To do that we parse the two objects (`original` and `mutant`) that hold all the informations from the pdb file to find the atoms of each structure by acquiring the following information of each atom:

e.g.

    NAME RESIDUE_NAME RESID
     CA      TYR       369
    
    
We can easily obtain these information, by using the methods `names`, `resnames` and `resnums` on each of the two structures. We hold the results of the parsign in two lists (`atoms_original` and `atoms_mutant`). Finally, we find the common atoms between the two lists by fisrt converting them into python sets and then calculating their intersection. We hold the results in the `common_atoms` and observe that there are 1454 common atoms. 

In [6]:
atoms_original = [(name, residue, resid) for name, residue, resid in zip(original.atoms.names,
                                                                         original.atoms.resnames,
                                                                         original.atoms.resnums)]
atoms_mutant = [(name, residue, resid) for name, residue, resid in zip(mutant.atoms.names, 
                                                                       mutant.atoms.resnames,
                                                                       mutant.atoms.resnums)]

common_atoms = list(set(atoms_original) & set(atoms_mutant))

print(f'There are {len(common_atoms)} common atoms.')

There are 1454 common atoms.


# RMSD over all atoms

We calculate the RMSD over **all common atoms**, between the two structures. 

In [7]:
original_coors = np.array([original.select_atoms(f"name {name} and resname {residue} and resid {resid}").positions[0].tolist() 
                           for name, residue, resid in common_atoms])
mutant_coors = np.array([mutant.select_atoms(f"name {name} and resname {residue} and resid {resid}").positions[0].tolist() 
                         for name, residue, resid in common_atoms])

In [8]:
RMSD_all = rms.rmsd(original_coors,  # coordinates to align
                    mutant_coors,  # reference coordinates
                    center=True,  # subtract the center of geometry
                    superposition=True)  # superimpose coordinates

print(f'cRMSD using all atoms is: {RMSD_all}')

cRMSD using all atoms is: 0.6302421980565904


# RMSD over Ca atoms

We calculate the RMSD over the **common $C_{\alpha}$ atoms**, between the two structures. 

In [9]:
ca_original = np.array([original.select_atoms(f"name {name} and resname {residue} and resid {resid}").positions[0].tolist() 
                 for name, residue, resid in common_atoms 
                 if name == 'CA'])
ca_mutant = np.array([mutant.select_atoms(f"name {name} and resname {residue} and resid {resid}").positions[0].tolist() 
                 for name, residue, resid in common_atoms 
                 if name == 'CA']) 


In [10]:
RMSD_Ca = rms.rmsd(ca_original,  # coordinates to align
                   ca_mutant,  # reference coordinates
                   center=True,  # subtract the center of geometry
                   superposition=True)  # superimpose coordinates

print(f'cRMSD using all atoms is: {RMSD_Ca}')

cRMSD using all atoms is: 0.2925915300664362


We note that cRMSD is larger when we use all the atoms than when we use only $C_{\alpha}$ atoms. This is expected since $C_{\alpha}$ atoms consist the main element of the proteins backbone and are very conserved and steady structures. Moreover, there are more degrees of freedom when we use all the atoms to calculate the RMSD. 